In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import plotly.graph_objects as go
from sklearn.mixture import GaussianMixture
import plotly.express as px

from clusterbuster import parse_report, plot_clusters, gtype_gmm, confidence_ellipse, plot_gmm, calculate_maf

Some info on the values:

R values are the intensities. R for a sample is the sum of normalized X and Y (R= X+ Y)
Theta is the B allele frequency and is calculated by: (2/pi)*arctan(normalized Y/ normalized X)

BAF = Y / (X + Y)
LRR = log2( (X + Y)sampleOfInterest / (X+Y)baselineSample)

In [2]:
# first, need to subset the huge report to just snps of interest
# report = pd.read_csv('F4SEttehadieh_P1_Genotyping_Report.txt', engine='c', sep='\t', dtype={'Chr':str, 'position':int})
# split by chromosome
# for chrom in report.Chr.unique():
#     report[report.Chr==chrom].to_csv(f'data/chr{chrom}_report.csv', header=True, index=False)

# report[report['GenTrain Score']<0.15]
# snps_df = pd.read_csv('prototyping/snps_of_interest_first_data.txt', engine='c')
# report[report.Name.isin(snps_df.Name)].to_csv('data/cb_test_snps.txt', sep='\t', index=False)

# check_header = pd.read_csv('../data/chr22_report.csv', nrows=0)
# header_list = ['Index', 'Name', 'Address', 'Chr', 'Position', 'GenTrain Score','Frac A', 'Frac C', 'Frac G', 'Frac T']
# all(elem in check_header.columns for elem in header_list)
# check_header.columns


In [3]:
report_in = pd.read_csv('../data/chr22_report.csv', engine='c', dtype={'Chr':str, 'position':int})

report = parse_report(report_in, flag_maf=0.01, flag_gentrain=0.5)
# test_report.keys()
cb_df = report['clusterbuster_df']
snps_df = report['flagged_snps']
flagged = snps_df[snps_df.maf_flag & snps_df.gentrain_flag]

total_flagged = snps_df[snps_df.maf_flag | snps_df.gentrain_flag].reset_index()
both_flagged = snps_df[snps_df.maf_flag & snps_df.gentrain_flag].reset_index()
maf_flagged = snps_df.loc[snps_df.maf_flag].reset_index()
gentrain_flagged = snps_df.loc[snps_df.gentrain_flag].reset_index()

In [4]:
report['clusterbuster_df']

idx,IID,1KG_22_19435270.GType,1KG_22_19766749.GType,1KG_22_24530438.GType,1KG_22_24909316.GType,1KG_22_26286764.GType,1KG_22_26298671.GType,1KG_22_30857665.GType,1KG_22_36681998.GType,1KG_22_38494134.GType,...,Variant61228.R,Variant61231.R,Variant61234.R,Variant61237.R,Variant61242.R,Variant61244.R,Variant61245.R,Variant61246.R,Variant61252.R,Variant61253.R
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.430500,0.361874,0.540405,0.626420,0.957749,0.733668,0.795464,0.701761,0.470309,0.569939
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.415114,0.329952,0.549609,0.571324,0.932040,0.671134,0.730768,0.735666,0.476564,0.603078
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.379468,0.420835,0.541583,0.608452,0.976639,0.641460,0.726527,0.748987,0.439781,0.518233
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.462127,0.387539,0.521502,0.639255,0.965216,0.699769,0.735392,0.791597,0.410290,0.626747
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.469232,0.366763,0.586054,0.579106,1.041958,0.677856,0.758849,0.712157,0.468440,0.582567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.359078,0.360436,0.449827,0.581284,1.042775,0.702279,0.757134,0.718126,0.447970,0.575152
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.365155,0.359822,0.513683,0.557322,0.872211,0.649222,0.692903,0.726661,0.427236,0.530507
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.329993,0.388541,0.518266,0.577355,0.891683,0.711396,0.780114,0.710056,0.407510,0.513913
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.354541,0.322347,0.421428,0.515936,0.894079,0.712438,0.747326,0.663459,0.403457,0.479983


In [10]:
# selected from user input
selected_snp = 'JHU_22.26156400'
# selected_snp = 'JHU_22.27408320'

geno_col = selected_snp + ".GType"
theta_col = selected_snp + ".Theta"
r_col = selected_snp + ".R"

gtypes = list(cb_df[geno_col].unique())

snp_for_plot_df = cb_df[['IID', theta_col, r_col, geno_col]].copy()
snp_for_plot_df.columns = snp_for_plot_df.columns.str.replace(f'{selected_snp}.', '', regex=False)

df = snp_for_plot_df.copy()
x_col, y_col = 'Theta', 'R'
gtypes_list = list(df.GType.unique())

df = snp_for_plot_df.copy()
x_col, y_col, gtype_col = 'Theta', 'R', 'GType'
gtypes_list = (df.GType.unique())

clusters_fig = plot_clusters(df=df, x_col=x_col, y_col=y_col, gtype_col=gtype_col, snpid=selected_snp)

clusters_fig.show()

In [11]:
# now train a gmm and plot reclustered variants
# get genotypes and number for choosing number of components
gtypes_for_gmm = list(cb_df[geno_col].unique())
gtypes_for_gmm.remove('NC')
n_components = len(gtypes_for_gmm)

snp_for_gmm_df = cb_df[['IID', theta_col, r_col, geno_col]].copy()
snp_for_gmm_df.columns = snp_for_gmm_df.columns.str.replace(f'{selected_snp}.','', regex=False)
snp_for_gmm_df

idx,IID,Theta,R,GType
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,0.062027,0.600773,AA
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,0.081050,0.728735,AA
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,0.086248,0.668563,AA
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,0.100073,0.687123,AA
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,0.073081,0.624279,AA
...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,0.110191,0.643409,AA
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,0.082090,0.650478,AA
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,0.080408,0.673840,AA
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,0.135242,0.652273,AA


In [12]:


# def gtype_gmm(snp_theta_r_df, n_components):
    
#     X = snp_theta_r_df[['Theta','R']].copy()

#     gmm = GaussianMixture(
#         n_components=n_components,
#         covariance_type="diag",
#         random_state = 10).fit(X)
    
#     labels = gmm.predict(X)

#     out_dict = {
#         'gmm': gmm,
#         'X': X,
#         'y_pred': labels
#     }

#     return out_dict


#     import numpy as np


# def confidence_ellipse(x, y, n_std=1.96, size=100):
#     """
#     Get the covariance confidence ellipse of *x* and *y*.
#     Parameters
#     ----------
#     x, y : array-like, shape (n, )
#         Input data.
#     n_std : float
#         The number of standard deviations to determine the ellipse's radiuses.
#     size : int
#         Number of points defining the ellipse
#     Returns
#     -------
#     String containing an SVG path for the ellipse
    
#     References (H/T)
#     ----------------
#     https://matplotlib.org/3.1.1/gallery/statistics/confidence_ellipse.html
#     https://community.plotly.com/t/arc-shape-with-path/7205/5
#     """
#     cov = np.cov(x, y)
#     pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
#     # Using a special case to obtain the eigenvalues of this
#     # two-dimensionl dataset.
#     ell_radius_x = np.sqrt(1 + pearson)
#     ell_radius_y = np.sqrt(1 - pearson)
#     theta = np.linspace(0, 2 * np.pi, size)
#     ellipse_coords = np.column_stack([ell_radius_x * np.cos(theta), ell_radius_y * np.sin(theta)])
    
#     # Calculating the stdandard deviation of x from
#     # the squareroot of the variance and multiplying
#     # with the given number of standard deviations.
#     x_scale = np.sqrt(cov[0, 0]) * n_std
#     x_mean = np.mean(x)

#     # calculating the stdandard deviation of y ...
#     y_scale = np.sqrt(cov[1, 1]) * n_std
#     y_mean = np.mean(y)
  
#     translation_matrix = np.tile([x_mean, y_mean], (ellipse_coords.shape[0], 1))
#     rotation_matrix = np.array([[np.cos(np.pi / 4), np.sin(np.pi / 4)],
#                                 [-np.sin(np.pi / 4), np.cos(np.pi / 4)]])
#     scale_matrix = np.array([[x_scale, 0],
#                             [0, y_scale]])
#     ellipse_coords = ellipse_coords.dot(rotation_matrix).dot(scale_matrix) + translation_matrix
        
#     path = f'M {ellipse_coords[0, 0]}, {ellipse_coords[0, 1]}'
#     for k in range(1, len(ellipse_coords)):
#         path += f'L{ellipse_coords[k, 0]}, {ellipse_coords[k, 1]}'
#     path += ' Z'
#     return path


# def plot_gmm(df, x_col, y_col, gtype_col, gmm, snpid, n_cont=5):

#     d3 = px.colors.qualitative.D3

#     cmap = {
#         'AA': d3[0],
#         'AB': d3[1],
#         'BA': d3[1],
#         'BB': d3[2],
#         'NC': d3[3]
#     }

#     gtypes_list = (df[gtype_col].unique())


#     fig = go.Figure()

#     for gtype in gtypes_list:
#         df_ = df.loc[df[gtype_col]==gtype]
#         x = df_.loc[:,x_col]
#         y = df_.loc[:,y_col]
#         color = cmap[gtype]
#         fig.add_trace(
#             go.Scatter(
#                 x=x, y=y, 
#                 mode="markers",
#                 name=gtype,
#                 marker = {'color':color},
#                 hovertemplate="Genotype=%s<br>Theta=%%{x}<br>R=%%{y}<extra></extra>"% gtype
#                 ))

#         fig.update_layout(title_text=f'{snpid} Cluster Plot')
#         fig.update_layout(legend_title_text = 'Genotype')
#         fig.update_xaxes(title_text=x_col)
#         fig.update_yaxes(title_text=y_col)

        
#         for n_cont in range(1, n_cont):
#             fig.add_shape(type='path',
#             path=confidence_ellipse(x, y, n_std=1.96*n_cont, size=100), 
#             line_color=color,
#             fillcolor=color,
#             opacity=0.2)
    
#     return fig



In [13]:
gmm_out = gtype_gmm(snp_theta_r_df=snp_for_gmm_df, n_components=n_components)

In [14]:
# handle missing
missing_df = pd.DataFrame()

missing_snp = snp_for_gmm_df.loc[snp_for_gmm_df.Theta.isna() | snp_for_gmm_df.R.isna() | snp_for_gmm_df.GType.isna()].copy()
missing_snp.loc[:,'snpid'] = selected_snp
missing_df = missing_df.append(missing_snp)

missing_df

ValueError: cannot set a frame with no defined index and a scalar

In [14]:
snp_for_gmm_df.loc[~snp_for_gmm_df.Theta.isna() & ~snp_for_gmm_df.R.isna() & ~snp_for_gmm_df.GType.isna()].copy()

idx,IID,Theta,R,GType
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,0.075856,1.086711,AA
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,0.055597,1.006523,AA
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,0.057294,1.063352,AA
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,0.082261,1.077064,AA
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,0.055115,1.016364,AA
...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,0.058748,1.021547,AA
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,0.073851,1.108173,AA
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,0.044587,0.990201,AA
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,0.064544,1.156772,AA


In [18]:
cb_df
snps_df

,snpid,maf,Chr,Position,gentrain_score,Frac_A,Frac_C,Frac_G,Frac_T,maf_flag,gentrain_flag
0,1KG_22_19435270,0.000000,22,19435270,0.867919,0.267327,0.376238,0.222772,0.133663,True,False
1,1KG_22_19766749,0.000000,22,19766749,0.909582,0.272277,0.198020,0.316832,0.212871,True,False
2,1KG_22_24530438,0.000000,22,24530438,0.868893,0.212871,0.237624,0.297030,0.252475,True,False
3,1KG_22_24909316,0.000000,22,24909316,0.828910,0.262376,0.202970,0.257426,0.277228,True,False
4,1KG_22_26286764,0.000000,22,26286764,0.909582,0.183168,0.257426,0.346535,0.212871,True,False
...,...,...,...,...,...,...,...,...,...,...,...
25990,Variant61244,0.000000,22,51065646,0.884393,0.193548,0.290323,0.338710,0.177419,True,False
25991,Variant61245,0.000000,22,51065647,0.902295,0.193548,0.290323,0.338710,0.177419,True,False
25992,Variant61246,0.000000,22,51065647,0.892761,0.185484,0.338710,0.290323,0.185484,True,False
25993,Variant61252,0.000000,22,51065808,0.829425,0.217742,0.338710,0.370968,0.072581,True,False


In [15]:
gmm_plot_df = gmm_out['X']
gmm_plot_df.loc[:,'GType'] = gmm_out['y_pred']
gmm = gmm_out['gmm']


# pos = gmm.means_[1]
# width, height = 2 * np.sqrt(covariances[1])
# for gtype in gmm_plot_df.GType.unique():


cluster_means = {gtype:gmm_plot_df.loc[gmm_plot_df.GType==gtype,'Theta'].mean() for gtype in gmm_plot_df.GType.unique()}

gtype_order = ['AA','AB','BB']
gtypes_for_gmm_ordered = [gtype for gtype in gtype_order if gtype in gtypes_for_gmm]
ordered_cluster_means = dict(sorted(cluster_means.items(), key=lambda item: item[1]))
gmm_gtype_map = {list(ordered_cluster_means.keys())[i]:gtype for i, gtype in enumerate(gtypes_for_gmm_ordered)}

gmm_plot_df.loc[:,'gtype_out'] = gmm_plot_df.loc[:,'GType'].replace(gmm_gtype_map)
gmm_plot_df.loc[:,'original_gtype'] = snp_for_gmm_df.loc[:,'GType']


In [16]:
gmm_plot_df

idx,Theta,R,GType,gtype_out,original_gtype
0,0.062027,0.600773,1,AA,AA
1,0.081050,0.728735,1,AA,AA
2,0.086248,0.668563,1,AA,AA
3,0.100073,0.687123,1,AA,AA
4,0.073081,0.624279,1,AA,AA
...,...,...,...,...,...
91,0.110191,0.643409,1,AA,AA
92,0.082090,0.650478,1,AA,AA
93,0.080408,0.673840,1,AA,AA
94,0.135242,0.652273,1,AA,AA


In [37]:
plot_gmm(df=gmm_plot_df, x_col='Theta', y_col='R', gtype_col='gtype_out', gmm=gmm, snpid=selected_snp, n_cont=5)

In [48]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = np.random.randn(200)-2
x2 = np.random.randn(200)
x3 = np.random.randn(200)+2
x4 = np.random.randn(200)+4

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.1, .25, .5, 1])
fig.show()

In [49]:
cb_df

idx,IID,1KG_22_19435270.GType,1KG_22_19766749.GType,1KG_22_24530438.GType,1KG_22_24909316.GType,1KG_22_26286764.GType,1KG_22_26298671.GType,1KG_22_30857665.GType,1KG_22_36681998.GType,1KG_22_38494134.GType,...,Variant61228.R,Variant61231.R,Variant61234.R,Variant61237.R,Variant61242.R,Variant61244.R,Variant61245.R,Variant61246.R,Variant61252.R,Variant61253.R
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.430500,0.361874,0.540405,0.626420,0.957749,0.733668,0.795464,0.701761,0.470309,0.569939
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.415114,0.329952,0.549609,0.571324,0.932040,0.671134,0.730768,0.735666,0.476564,0.603078
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.379468,0.420835,0.541583,0.608452,0.976639,0.641460,0.726527,0.748987,0.439781,0.518233
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.462127,0.387539,0.521502,0.639255,0.965216,0.699769,0.735392,0.791597,0.410290,0.626747
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.469232,0.366763,0.586054,0.579106,1.041958,0.677856,0.758849,0.712157,0.468440,0.582567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.359078,0.360436,0.449827,0.581284,1.042775,0.702279,0.757134,0.718126,0.447970,0.575152
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.365155,0.359822,0.513683,0.557322,0.872211,0.649222,0.692903,0.726661,0.427236,0.530507
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.329993,0.388541,0.518266,0.577355,0.891683,0.711396,0.780114,0.710056,0.407510,0.513913
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.354541,0.322347,0.421428,0.515936,0.894079,0.712438,0.747326,0.663459,0.403457,0.479983


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

gtypes_map = {
    'AA':'0',
    'AB':'1',
    'BA':'1',
    'BB':'2',
    }

gtypes_map_rev = {
    '0':'AA',
    '1':'AB',
    '2':'BB'
}

# samples to train and test on
to_gmm_samples = snp_for_gmm_df.loc[snp_for_gmm_df[geno_col]!='NC']
# to_gmm_samples.loc[:,'gtype'] = np.where(to_gmm_samples.loc[:, geno_col]=='BA', 'AB', to_gmm_samples.loc[:, geno_col])

# samples to predict
to_pred_samples = snp_for_gmm_df.loc[snp_for_gmm_df[geno_col]=='NC']

X = to_gmm_samples.loc[:,[theta_col, r_col]]
y = np.where(to_gmm_samples.loc[:, geno_col]=='BA', 'AB', to_gmm_samples.loc[:, geno_col])
y = to_gmm_samples.loc[:,geno_col].replace(gtypes_map)

x_pred = to_pred_samples.loc[:,[theta_col, r_col]]
# n_classes = len(np.unique(y))
classes = np.unique(y)

gmm = GaussianMixture(
        n_components=n_components,
        covariance_type="diag",
        random_state = 10).fit(X)

gmm.means_ = np.array([X[y == i].mean(axis=0) for i in classes])

gmm.fit(X)
y_pred = gmm.predict(x_pred)
y_pred

In [36]:
# import math
# position = gmm.means_
# covariances = gmm.covariances_
# w = gmm.weights_

# position[0][0]

# def ellipse(x_center=0, y_center=0, ax1 = [1, 0],  ax2 = [0,1], a=1, b =1,  N=100):
#    # x_center, y_center the coordinates of ellipse center
#    # ax1 ax2 two orthonormal vectors representing the ellipse axis directions
#    # a, b the ellipse parameters
#    if np.linalg.norm(ax1) != 1 or np.linalg.norm(ax2) != 1:
#        raise ValueError('ax1, ax2 must be unit vectors')
#    if  abs(np.dot(ax1, ax2)) > 1e-06:
#        raise ValueError('ax1, ax2 must be orthogonal vectors')
#    #rotation matrix   
#    R = np.array([ax1, ax2]).T
#    if np.linalg.det(R) <0: 
#        raise ValueError("the det(R) must be positive to get a  positively oriented ellipse reference frame")
#    t = np.linspace(0, 2*math.pi, N)
#    #ellipse parameterization with respect to a system of axes of directions a1, a2
#    xs = a * math.cos(t)
#    ys = b * math.sin(t)
   
#    # coordinate of the  ellipse points with respect to the system of axes [1, 0], [0,1] with origin (0,0)
#    xp, yp = np.dot(R, [xs, ys])
#    x = xp + x_center 
#    y = yp + y_center
#    return x, y


# ellipse(x_center=position[0], y_center=position[1])

0.896737899999998

In [11]:
# snp_list = gtype_df.idx
# snps_of_interest = list()
# count=0
# for snp in snp_list:
#     if set(['NC','AA','BB']).issubset(list(GType_transposed_df[snp].unique())):
#         vc = GType_transposed_df[snp].value_counts()
#         # print(vc)
#         snps_of_interest.append(snp)
#         count+=1
       
# print(count)

39


In [13]:
# # flag_df[(flag_df.maf>0.03) & (flag_df.maf<0.5)
# snps_of_interest_df = pd.DataFrame({'snp':snps_of_interest})
# snps_of_interest_df.loc[:,'snp'] = snps_of_interest_df.loc[:,'snp'].str.replace('.GType','',regex=False) 
# snps_of_interest_df.to_csv('../data/testing_snps_of_interest.csv', index=False)

In [26]:
# Chop out to GType, Score and Theta dataframes.
flag_maf=0.01
flag_gentrain=0.5
report = report_in.copy()
Name_df = report[['Name']].copy()
GType_df = report.loc[:, report.columns.str.endswith(".GType")]
Theta_df = report.loc[:, report.columns.str.endswith(".Theta")]
R_df = report.loc[:, report.columns.str.endswith(".R")]

Name_df['Name.GType'] = Name_df.loc[:,'Name'] + ".GType"
Name_df['Name.Theta'] = Name_df.loc[:,'Name'] + ".Theta"
Name_df['Name.R'] = Name_df.loc[:,'Name'] + ".R"

# Merge names plus data types.
Name_GType_df = pd.concat([Name_df, GType_df], axis=1).rename(columns={"Name.GType":"idx"}).drop(columns=['Name', 'Name.Theta', 'Name.R'])
Name_GType_df.columns = Name_GType_df.columns.str.replace(".GType", "", regex=False)
Name_GType_df_final = Name_GType_df.set_index('idx')

Name_Theta_df = pd.concat([Name_df, Theta_df], axis=1).rename(columns={"Name.Theta": "idx"}).set_index('idx').drop(columns=['Name', 'Name.GType', 'Name.R'])
Name_Theta_df.columns = Name_Theta_df.columns.str.replace(".Theta", "", regex=False)

Name_R_df = pd.concat([Name_df, R_df], axis=1).rename(columns={"Name.R":"idx"}).set_index('idx').drop(columns=['Name', 'Name.Theta', 'Name.GType'])
Name_R_df.columns = Name_R_df.columns.str.replace(".R", "", regex=False)

#  Transpose the data frames and make the names of the variants plus the suffixes the columns.
GType_transposed_df = Name_GType_df_final.transpose()
Theta_transposed_df = Name_Theta_df.transpose()
R_transposed_df = Name_R_df.transpose()

# Smash everything together and get ready for plotting.
temp_df = GType_transposed_df.merge(Theta_transposed_df, left_index=True, right_index=True)
clusterbuster_df = temp_df.merge(R_transposed_df, left_index=True, right_index=True)
clusterbuster_out = clusterbuster_df.reset_index().rename(columns={'index':'IID'})

# get gentrain scores

gtrain_scores_df = report_in.loc[:,['Name', 'Chr', 'Position','GenTrain Score','Frac A', 'Frac C', 'Frac G', 'Frac T']]
gtrain_scores_df.columns = ['snpid','Chr','Position','gentrain_score','Frac_A','Frac_C','Frac_G','Frac_T']
# calculate maf
gtype_df = Name_GType_df.copy()
gtype_df.loc[:,'snpid'] = gtype_df.loc[:,'idx'].str.replace(".GType", "", regex=False)
gtype_to_maf = gtype_df.drop(columns=['idx'])
maf_scores_df = calculate_maf(gtype_to_maf)
flag_df = maf_scores_df.merge(gtrain_scores_df, how='inner', on='snpid')
flag_df.loc[:,'maf_flag'] = np.where(flag_df.maf<flag_maf, True, False)
flag_df.loc[:,'gentrain_flag'] = np.where(flag_df.gentrain_score<flag_gentrain, True, False)

missing_df = pd.DataFrame()


out_dict = {
    'clusterbuster_df': clusterbuster_out,
    'flagged_snps': flag_df, 
}

In [36]:
# Name_Theta_df[Name_Theta_df.isna(axis=1)]

Name_Theta_df[Name_Theta_df.isnull().any(axis=1)]

,FS04821313_FO0000275_FUS_R495*_A01_17_A01,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,FS04821316_FO0000176_ANG_K41I_A04_17_D01,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,FS04821318_FO0000200_PFN1_C71G_A01_17_F01,FS04821319_FO0000212_HNRNPA1_D262N_A01_17_G01,FS04821320_FO0000224_CSF1R_E633K_A01_17_H01,FS04821321_FO0000276_FUS_R495*_A04_17_A02,FS04821322_FO0000153_MAPT_R317W_A10_17_B02,...,FS04821399_FO0000166_APP_A673T_A10_17_C03,FS04821400_FO0000167_APP_A673T_B02_17_C04,FS04821401_FO0000168_APP_A673T_B11_17_C05,FS04821402_FO0000169_APP_A673T_C03_17_C06,FS04821403_FO0000170_APP_A673T_C04_17_C07,FS04821404_FO0000171_APP_A673T_D09_17_C08,FS04821405_FO0000172_APP_A673T_D12_17_C09,FS04821406_FO0000173_APP_A673T_F06_17_C10,FS04821407_FO0000174_APP_A673T_H08_17_C11,PositiveControl_1_NA_NA
idx,,,,,,,,,,,,,,,,,,,,,
22:19263188-TC.Theta,0.016549,0.016549,0.034746,0.005748,0.013034,0.028882,0.036758,0.009955,0.027756,0.026663,...,0.017424,0.030915,0.026956,0.011431,0.021527,0.011655,0.035450,0.039059,0.029760,0.050559
22:21354461-GA.Theta,0.958369,0.881275,0.842105,NaN,0.900391,0.901443,0.901834,0.950166,0.885677,0.955232,...,0.856874,0.896709,0.903836,0.946666,0.937902,0.927664,0.946430,0.931566,0.867493,0.524616
22:21384250-AG.Theta,0.040352,0.093047,0.065083,0.079220,0.063334,0.056559,0.088066,0.097245,0.146142,0.121591,...,0.101050,0.109888,0.090588,0.062786,0.094529,0.121504,0.117310,0.072148,0.136329,0.434891
22:21984279-CA.Theta,0.936341,0.912554,0.924742,0.921270,0.957253,0.954720,0.900188,0.946820,0.911227,0.937269,...,0.880905,0.919169,0.942667,0.971814,0.899858,0.977350,0.914369,0.905627,0.905158,0.607808
22:24481101-AG.Theta,0.027583,0.029278,0.020352,0.033576,0.017459,0.029715,0.028005,0.025697,0.014463,0.022326,...,0.030915,0.023189,0.010187,0.016286,0.017335,0.018646,0.020970,0.031749,0.016931,0.029535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs9625374.Theta,0.578136,0.577489,0.562329,0.567274,0.587530,0.578070,0.591982,0.696036,0.603432,0.613582,...,0.618050,0.568534,0.558752,0.603907,0.591012,0.579917,0.589297,0.626615,0.603600,0.019538
rs9626059.Theta,0.959312,0.956198,0.945253,0.927606,0.948385,0.953619,0.958424,0.931868,0.926692,0.931908,...,0.911472,0.917925,0.952390,0.973589,0.954016,0.950417,0.937735,0.960297,0.951617,0.920231
rs9628047.Theta,0.515638,0.536284,0.560650,0.539354,0.543270,0.538346,0.570527,0.482153,0.550294,0.578773,...,0.520268,0.537218,0.557735,0.506185,0.537496,0.575556,0.550700,0.554986,0.564009,0.518252


In [31]:
Name_Theta_df

,FS04821313_FO0000275_FUS_R495*_A01_17_A01,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,FS04821316_FO0000176_ANG_K41I_A04_17_D01,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,FS04821318_FO0000200_PFN1_C71G_A01_17_F01,FS04821319_FO0000212_HNRNPA1_D262N_A01_17_G01,FS04821320_FO0000224_CSF1R_E633K_A01_17_H01,FS04821321_FO0000276_FUS_R495*_A04_17_A02,FS04821322_FO0000153_MAPT_R317W_A10_17_B02,...,FS04821399_FO0000166_APP_A673T_A10_17_C03,FS04821400_FO0000167_APP_A673T_B02_17_C04,FS04821401_FO0000168_APP_A673T_B11_17_C05,FS04821402_FO0000169_APP_A673T_C03_17_C06,FS04821403_FO0000170_APP_A673T_C04_17_C07,FS04821404_FO0000171_APP_A673T_D09_17_C08,FS04821405_FO0000172_APP_A673T_D12_17_C09,FS04821406_FO0000173_APP_A673T_F06_17_C10,FS04821407_FO0000174_APP_A673T_H08_17_C11,PositiveControl_1_NA_NA
idx,,,,,,,,,,,,,,,,,,,,,
1KG_22_19435270.Theta,0.944591,0.924225,0.952862,0.935011,0.928478,0.937088,0.918155,0.929431,0.923251,0.941469,...,0.930244,0.932926,0.925883,0.951190,0.912769,0.946002,0.915300,0.953285,0.931741,0.925320
1KG_22_19766749.Theta,0.961539,0.983606,0.978408,0.990258,0.958904,0.982706,0.975860,0.972207,0.971266,0.973653,...,0.982327,0.959679,0.971349,0.986706,0.967694,0.991431,0.979121,0.957424,0.967373,0.970041
1KG_22_24530438.Theta,0.935055,0.929540,0.936355,0.943631,0.935300,0.938723,0.929532,0.945230,0.938823,0.917506,...,0.899698,0.896716,0.932028,0.943460,0.952184,0.939595,0.946390,0.918618,0.907242,0.953646
1KG_22_24909316.Theta,0.988443,0.998173,0.988477,0.973814,0.980671,0.989722,0.981267,0.981476,0.979087,0.987872,...,0.966775,0.979639,0.980440,0.973590,0.984073,0.976006,0.973179,0.970715,0.972820,0.978037
1KG_22_26286764.Theta,0.033041,0.034184,0.030396,0.017288,0.020574,0.012635,0.030192,0.028967,0.015226,0.023375,...,0.045140,0.018201,0.023581,0.005587,0.040401,0.027224,0.040619,0.035331,0.005580,0.023618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Variant61244.Theta,0.962964,0.931996,0.944310,0.961858,0.986630,0.967963,0.954373,0.956936,0.951270,0.974294,...,0.930658,0.928149,0.962309,0.975746,0.934768,0.949947,0.879090,0.964052,0.968252,0.829358
Variant61245.Theta,0.989370,0.948174,0.984073,0.984382,0.971789,0.966652,0.950295,0.946203,0.954777,0.971629,...,0.972714,0.948279,0.930362,0.984697,0.962102,0.984710,0.980356,0.955761,0.972587,0.875520
Variant61246.Theta,0.932300,0.943781,0.984699,0.974981,0.954198,0.965815,0.990888,0.920727,1.000000,0.968389,...,0.962161,0.958462,0.971174,0.968120,0.958886,0.955142,0.940754,0.951538,0.991615,0.895762


In [35]:
clusterbuster_out

idx,IID,1KG_22_19435270.GType,1KG_22_19766749.GType,1KG_22_24530438.GType,1KG_22_24909316.GType,1KG_22_26286764.GType,1KG_22_26298671.GType,1KG_22_30857665.GType,1KG_22_36681998.GType,1KG_22_38494134.GType,...,Variant61228.R,Variant61231.R,Variant61234.R,Variant61237.R,Variant61242.R,Variant61244.R,Variant61245.R,Variant61246.R,Variant61252.R,Variant61253.R
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.430500,0.361874,0.540405,0.626420,0.957749,0.733668,0.795464,0.701761,0.470309,0.569939
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.415114,0.329952,0.549609,0.571324,0.932040,0.671134,0.730768,0.735666,0.476564,0.603078
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.379468,0.420835,0.541583,0.608452,0.976639,0.641460,0.726527,0.748987,0.439781,0.518233
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.462127,0.387539,0.521502,0.639255,0.965216,0.699769,0.735392,0.791597,0.410290,0.626747
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.469232,0.366763,0.586054,0.579106,1.041958,0.677856,0.758849,0.712157,0.468440,0.582567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.359078,0.360436,0.449827,0.581284,1.042775,0.702279,0.757134,0.718126,0.447970,0.575152
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.365155,0.359822,0.513683,0.557322,0.872211,0.649222,0.692903,0.726661,0.427236,0.530507
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.329993,0.388541,0.518266,0.577355,0.891683,0.711396,0.780114,0.710056,0.407510,0.513913
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.354541,0.322347,0.421428,0.515936,0.894079,0.712438,0.747326,0.663459,0.403457,0.479983
